# For  LDM-503-EFDb  

Initialize EFDs Summit and USDF

In [1]:
from lsst_efd_client import EfdClient, resample

client = EfdClient('summit_efd')
client.output = 'dataframe'
cl=client.influx_client



usdf_client = EfdClient('usdf_efd')
usdf_client.output = 'dataframe'
usdf_cl=usdf_client.influx_client


In [2]:
## Pick five topics

In [28]:
import random
async def selectTopics(pick):
    topics = await client.get_topics()

    selected_topics = []
    loc = "summit"

    day = '2023-05-20'

    # want to select topics randomly but with messages so randomize all indexes 
    randoms = random.sample(range(0,len(topics)),len(topics))
    for r in randoms:
       topic = topics[r]
       if ('Test' in topic):
           next
       result = []
       if len(selected_topics) < 5:
           query=f'''SELECT * FROM "{topic}" WHERE time >  '{day}T00:00:00.000Z' and time < '{day}T00:00:30.000Z' '''
           result = await cl.query(query)

       if len(result) > 20:
           print (f"{topic} had {len(result)} messages in first 30 min of {day}")
           selected_topics.append(topic)
           if len(selected_topics) > 4:
             break
    print(f"Random selction of five '{loc}' topics {selected_topics} with messages on {day}")
    return selected_topics

In [4]:
## Utility function to compare reusults

In [18]:
def cmp(topic, result, sresult):
   #print (f"{topic} had {len(result)} messages - summit had {len(sresult)}")
   if len(result) != len(sresult):
        print (f"\033[91m  {topic} does not have the same number of results \033[0m")
   else:
        for k in sresult.keys():
            if result[k].all() != sresult[k].all():
                print (f"\033[91m {topic} {key} does not match \033[0m")


In [6]:
## Get dataframes and compare summit and USDF
Doing this day by day 

In [29]:
selected_topics = await selectTopics(5) # do this once at least

lsst.sal.MTM1M3.powerSupplyData had 1500 messages in first 30 min of 2023-05-20
lsst.sal.ATHexapod.logevent_heartbeat had 30 messages in first 30 min of 2023-05-20
lsst.sal.MTMount.generalPurposeGlycolWater had 100 messages in first 30 min of 2023-05-20
lsst.sal.MTMount.dynaleneCooling had 100 messages in first 30 min of 2023-05-20
lsst.sal.MTMount.elevationDrives had 471 messages in first 30 min of 2023-05-20
Random selction of five 'summit' topics ['lsst.sal.MTM1M3.powerSupplyData', 'lsst.sal.ATHexapod.logevent_heartbeat', 'lsst.sal.MTMount.generalPurposeGlycolWater', 'lsst.sal.MTMount.dynaleneCooling', 'lsst.sal.MTMount.elevationDrives'] with messages on 2023-05-20


In [33]:
# test gives large window 6 days

selected_topics = ['lsst.sal.MTM1M3.powerSupplyData', 
                   'lsst.sal.ATHexapod.logevent_heartbeat',
                   'lsst.sal.MTMount.generalPurposeGlycolWater', 
                   'lsst.sal.MTMount.dynaleneCooling',
                   'lsst.sal.MTMount.elevationDrives']
data = {}
for d in range(20,21):
    day = f"2023-06-{d}"
    day2 = day
    print(f" Checking {day}")
    for topic in selected_topics:
        query=f'''SELECT * FROM "{topic}" WHERE time >  '{day}T00:00:00.000Z' and time < '{day2}T23:59:59.000Z' limit 1300000'''
        result = await cl.query(query)
        usdf_result = await usdf_cl.query(query)
        cmp(topic, usdf_result, result)
        data[topic] = usdf_result
    

 Checking 2023-06-20
lsst.sal.MTM1M3.powerSupplyData had 1300000 messages - summit had 1300000
lsst.sal.ATHexapod.logevent_heartbeat had 64819 messages - summit had 64818
  lsst.sal.ATHexapod.logevent_heartbeat does not have the same number of results 
lsst.sal.MTMount.generalPurposeGlycolWater had 180915 messages - summit had 180915
lsst.sal.MTMount.dynaleneCooling had 180915 messages - summit had 180915
lsst.sal.MTMount.elevationDrives had 906688 messages - summit had 906688


In [ ]:
## Reliability 

In [35]:

for topic in selected_topics:
    seqnum = data[topic]['private_seqNum']
    count = 0
    i = 1 # see if the sequence increases mostly
    prev = seqnum[0]
    while i < len(seqnum):
        if (seqnum[i] < prev):
            #print (f"Reset at {i} seqnum : {prev}, {seqnum[i]}")
            count = count + 1
        i = i + 1
    percent = 100 * ((len(seqnum) - count) / len(seqnum))
    print(f"{topic} private_seqNum increases {percent}% of the sequence")
    
    

lsst.sal.MTM1M3.powerSupplyData private_seqNum increases 100.0% of the sequence
lsst.sal.ATHexapod.logevent_heartbeat private_seqNum increases 72.67776423579507% of the sequence
lsst.sal.MTMount.generalPurposeGlycolWater private_seqNum increases 90.85979603681287% of the sequence
lsst.sal.MTMount.dynaleneCooling private_seqNum increases 90.85979603681287% of the sequence
lsst.sal.MTMount.elevationDrives private_seqNum increases 90.84381838074398% of the sequence


In [ ]:
sday = '2023-06-1'     

for i in range(0,7):
    day = f"{sday}{i}"
    print(f"30 min on {day}")
    for topic in selected_topics:
        query=f'''SELECT * FROM "{topic}" WHERE time >  '{day}T00:00:00.000Z' and time < '{day}T00:00:30.000Z' '''
        result = await cl.query(query)
        usdf_result = await usdf_cl.query(query)
        cmp(topic, usdf_result, result)

